In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

day_obs = int(os.environ.get("SCHEDVIEW_DAY_OBS", datetime.date.today().strftime("%Y%m%d")))
earliest_sim = datetime.date.fromisoformat(os.environ.get("SCHEDVIEW_SIM_DATE", datetime.date(day_obs//10000, (day_obs%10000)//100, day_obs%100).isoformat()))

In [ ]:
import os
import sys

#sched_source = 'usdf'
sched_source = 'shared'
#sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.2.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.18.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.4.1.dev140+g532ec2c46")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.19.2.dev11+g44bf8dd")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/uranography")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
    case _:
        # Use the current environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
from IPython.display import display, HTML, Markdown
from collections import OrderedDict
import datetime
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
import boto3
import warnings
import astropy.units as u
from astropy.time import Time, TimezoneInfo
from urllib.parse import urlparse
from lsst.resources import ResourcePath
import yaml
from contextlib import redirect_stdout
import io
from erfa import ErfaWarning

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/shared/rubin_sim_data'
os.environ["AWS_PROFILE"] = "prenight"

In [ ]:
import schedview.collect.resources
import schedview.plot
import rubin_sim
from rubin_sim.sim_archive import vseqarchive
from schedview import DayObs
from rubin_sim.sim_archive.prenightindex import get_prenight_index
from schedview.plot.prenighttable import prenight_html_for_telescope

In [ ]:
telescopes = ('simonyi', 'auxtel')

In [ ]:
# Suppress benign warnings.

warnings.filterwarnings(
    "ignore",
    category=ErfaWarning,
    message=r".*ERFA function.*dubious year.*",
)

In [ ]:
prenights = {}
for telescope in telescopes:
    try:
        prenights[telescope] = get_prenight_index(day_obs, telescope, user='xreader', schema='vsmd', prenight_index_path=vseqarchive.PRENIGHT_INDEX_URL)
    except FileNotFoundError:
        pass

# Pre-night simulation archive overview with simulations including the `day_obs` of {{ params.day_obs }}

In [ ]:
for telescope in telescopes:
    display(Markdown(f'### {telescope} telescope'))
    if telescope in prenights and len(prenights[telescope]) > 0:
        display(HTML(prenight_html_for_telescope(day_obs, prenights[telescope])))
    else:
        display(Markdown(f"No prenights for {telescope}"))